In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision import models
from torchmetrics import Accuracy

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns

import random
import copy
import gc
import math

import warnings
warnings.filterwarnings('ignore')

# from lisa import LISA
from itertools import chain
from model import *
from pathlib import Path
from ranger import Ranger

In [34]:
from torch.utils.data import Dataset

class SubsetLISA(Dataset):
    """
    A custom dataset for the LISA subset, created using filtered image and label tensors.
    """
    def __init__(self, image_tensor_path, label_tensor_path,train:bool, transform=None):
        """
        Args:
            image_tensor_path (str): Path to the images tensor file.
            label_tensor_path (str): Path to the labels tensor file.
            transform (callable, optional): A function/transform to apply to the images.
        """
        self.images = torch.load(image_tensor_path)
        self.labels = torch.load(label_tensor_path)
        self.transform = transform
        self.train = train
        self._train_test_split()

        assert len(self.images) == len(self.labels), "Images and labels length mismatch"

    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        return len(self.labels)

    def __getitem__(self, index):
        """
        Args:
            index (int): Index of the sample.

        Returns:
            tuple: (image, label) where image is the input tensor and label is the target tensor.
        """
        image = self.images[index]
        target = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return image, target

    def _train_test_split(self, test_percent: float = 0.16):
        classes = {}
        for i, cl in enumerate(self.labels.numpy()):
            arr = classes.get(cl, [])
            arr.append(i)
            classes[cl] = arr

        train, test = [], []
        for cl, arr in classes.items():
            split_index = int(len(arr) * test_percent)
            test = test + arr[:split_index]
            train = train + arr[split_index:]

        sub = train if self.train else test
        self.images, self.labels = self.images[sub], self.labels[sub]

In [36]:
%run models.ipynb
%run utils.ipynb

In [110]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 50
batch_size = 8

In [40]:
normalize = transforms.Normalize(mean=[0.4563, 0.4076, 0.3895], std=[0.2298, 0.2144, 0.2259])

lisa_transforms = transforms.Compose([ transforms.ToPILImage(), transforms.Resize((224, 224)),transforms.ToTensor(),normalize])

# Paths to the saved subset tensors
image_tensor_path = "datasets/lisa-batches/subset_images.tensor"  # Replace with your actual file path
label_tensor_path = "datasets/lisa-batches/subset_labels2.tensor"  # Replace with your actual file path

# Initialize the dataset
train_dataset = SubsetLISA(image_tensor_path, label_tensor_path, train=True, transform = lisa_transforms)
test_dataset = SubsetLISA(image_tensor_path, label_tensor_path, train=False, transform = lisa_transforms)


In [42]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,) # num_workers=4,
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [69]:
model = resnet_18(filter='None', filter_layer=0)
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [71]:
def train_model(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [73]:
model_normal = train_model(model=model.to(device),epochs=epochs)

Epoch 1/100 - test accuracy: 48.05% and CE loss 1.48
Epoch 2/100 - test accuracy: 74.95% and CE loss 0.02
Epoch 3/100 - test accuracy: 77.87% and CE loss 0.04
Epoch 4/100 - test accuracy: 78.09% and CE loss 0.13
Epoch 5/100 - test accuracy: 88.61% and CE loss 0.24
Epoch 6/100 - test accuracy: 86.55% and CE loss 0.00
Epoch 7/100 - test accuracy: 85.14% and CE loss 0.01
Epoch 8/100 - test accuracy: 89.26% and CE loss 0.00
Epoch 9/100 - test accuracy: 92.62% and CE loss 0.00
Epoch 10/100 - test accuracy: 88.61% and CE loss 0.00
Epoch 11/100 - test accuracy: 92.62% and CE loss 0.00
Epoch 12/100 - test accuracy: 95.44% and CE loss 0.00
Epoch 13/100 - test accuracy: 86.66% and CE loss 0.00
Epoch 14/100 - test accuracy: 91.32% and CE loss 0.00
Epoch 15/100 - test accuracy: 96.64% and CE loss 0.00
Epoch 16/100 - test accuracy: 97.61% and CE loss 0.00
Epoch 17/100 - test accuracy: 97.83% and CE loss 0.00
Epoch 18/100 - test accuracy: 97.72% and CE loss 0.00
Epoch 19/100 - test accuracy: 97.72% 

In [74]:
# Check test set performance.
predictions, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for RESNET LISA Normal is: {(100 * test_acc):.2f}%")

Test accuracy for RESNET LISA Normal is: 98.26%


In [79]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_normal.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_normal.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_normal.pth


# L1 pre 

In [82]:
model = resnet_18(filter='None', filter_layer=0)
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

print(f"RESNET global sparsity = {compute_sparsity_resnet(model):.2f}%")

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
RESNET global sparsity = 0.00%


In [84]:
for name, module in model.named_modules():
    # prune 20% of weights/connections in for all hidden layaers-
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.2)
    
    # prune 10% of weights/connections for output layer-
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.1)

print(f"RESNET global sparsity = {compute_sparsity_resnet(model):.2f}%")

RESNET global sparsity = 19.99%


In [86]:
model_l1_unstructured = train_model(model = model.to(device), epochs = epochs)

Epoch 1/100 - test accuracy: 51.19% and CE loss 1.63
Epoch 2/100 - test accuracy: 69.63% and CE loss 0.04
Epoch 3/100 - test accuracy: 80.69% and CE loss 0.00
Epoch 4/100 - test accuracy: 77.11% and CE loss 0.00
Epoch 5/100 - test accuracy: 82.32% and CE loss 0.00
Epoch 6/100 - test accuracy: 85.25% and CE loss 0.00
Epoch 7/100 - test accuracy: 84.60% and CE loss 0.00
Epoch 8/100 - test accuracy: 85.03% and CE loss 0.02
Epoch 9/100 - test accuracy: 90.13% and CE loss 0.00
Epoch 10/100 - test accuracy: 92.30% and CE loss 0.00
Epoch 11/100 - test accuracy: 84.38% and CE loss 0.00
Epoch 12/100 - test accuracy: 88.83% and CE loss 0.00
Epoch 13/100 - test accuracy: 92.95% and CE loss 0.00
Epoch 14/100 - test accuracy: 96.53% and CE loss 0.00
Epoch 15/100 - test accuracy: 96.31% and CE loss 0.00
Epoch 16/100 - test accuracy: 95.23% and CE loss 0.00
Epoch 17/100 - test accuracy: 94.36% and CE loss 0.00
Epoch 18/100 - test accuracy: 93.06% and CE loss 0.00
Epoch 19/100 - test accuracy: 94.58% 

In [87]:
# Check test set performance.
predictions, labels = evaluate_model(model_l1_unstructured, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for RESNET LISA L1 pre is: {(100 * test_acc):.2f}%")

Test accuracy for RESNET LISA L1 pre is: 97.07%


In [93]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_l1_pre.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_l1_unstructured.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_l1_pre.pth


# Global Pre

In [112]:
model = resnet_18(filter='None', filter_layer=0)
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)
print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model):.2f}%")

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
RESNET-18 global sparsity = 0.00%


In [114]:
parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.bn1, 'weight'),
    (model.layer1[0].conv1, 'weight'),
    (model.layer1[0].bn1, 'weight'),
    (model.layer1[0].conv2, 'weight'),
    (model.layer1[0].bn2, 'weight'),
    (model.layer1[1].conv1, 'weight'),
    (model.layer1[1].bn1, 'weight'),
    (model.layer1[1].conv2, 'weight'),
    (model.layer1[1].bn2, 'weight'),
    (model.layer2[0].conv1, 'weight'),
    (model.layer2[0].bn1, 'weight'),
    (model.layer2[0].conv2, 'weight'),
    (model.layer2[0].bn2, 'weight'),
    (model.layer2[1].conv1, 'weight'),
    (model.layer2[1].bn1, 'weight'),
    (model.layer2[1].conv2, 'weight'),
    (model.layer2[1].bn2, 'weight'),
    (model.layer3[0].conv1, 'weight'),
    (model.layer3[0].bn1, 'weight'),
    (model.layer3[0].conv2, 'weight'),
    (model.layer3[0].bn2, 'weight'),
    (model.layer3[1].conv1, 'weight'),
    (model.layer3[1].bn1, 'weight'),
    (model.layer3[1].conv2, 'weight'),
    (model.layer3[1].bn2, 'weight'),
    (model.layer4[0].conv1, 'weight'),
    (model.layer4[0].bn1, 'weight'),
    (model.layer4[0].conv2, 'weight'),
    (model.layer4[0].bn2, 'weight'),
    (model.layer4[1].conv1, 'weight'),
    (model.layer4[1].bn1, 'weight'),
    (model.layer4[1].conv2, 'weight'),
    (model.layer4[1].bn2, 'weight'),
    (model.fc, 'weight')
)

prune_rates_global = [0.2, 0.3, 0.4, 0.5, 0.6]

def train_global_pruned(model, epochs):
    for iter_prune_round in range(1):
        print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
        
        # Prune layer-wise in a structured manner-
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method = prune.L1Unstructured,
            amount = prune_rates_global[iter_prune_round]
            
        )
    
        # Print current global sparsity level-
        print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model):.2f}%")
        
        
        # Fine-training loop-
        print("\nFine-tuning pruned model to recover model's performance\n")
        model.train()
        for epoch in range(epochs):
            for x_batch, y_batch in train_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                output = model(x_batch)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()
    
            # Evaluate model!
            if epochs%10==0:
                predictions, labels = evaluate_model(model, test_dataloader, device)
                test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
                print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [116]:
model_global = train_global_pruned(model = model.to(device), epochs = epochs)



Iterative Global pruning round = 1
RESNET-18 global sparsity = 20.00%

Fine-tuning pruned model to recover model's performance

Epoch 1/50 - test accuracy: 49.78% and CE loss 1.86
Epoch 2/50 - test accuracy: 74.95% and CE loss 0.40
Epoch 3/50 - test accuracy: 78.31% and CE loss 0.00
Epoch 4/50 - test accuracy: 81.24% and CE loss 0.01
Epoch 5/50 - test accuracy: 77.22% and CE loss 0.00
Epoch 6/50 - test accuracy: 74.30% and CE loss 0.00
Epoch 7/50 - test accuracy: 82.97% and CE loss 0.00
Epoch 8/50 - test accuracy: 85.25% and CE loss 0.00
Epoch 9/50 - test accuracy: 87.31% and CE loss 0.00
Epoch 10/50 - test accuracy: 88.29% and CE loss 0.00
Epoch 11/50 - test accuracy: 91.97% and CE loss 0.00
Epoch 12/50 - test accuracy: 96.10% and CE loss 0.00
Epoch 13/50 - test accuracy: 96.64% and CE loss 0.00
Epoch 14/50 - test accuracy: 96.96% and CE loss 0.00
Epoch 15/50 - test accuracy: 96.96% and CE loss 0.00
Epoch 16/50 - test accuracy: 96.96% and CE loss 0.00
Epoch 17/50 - test accuracy: 97

In [117]:
# Check test set performance.
predictions, labels = evaluate_model(model_global, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for RESNET LISA Global pre is: {(100 * test_acc):.2f}%")



Test accuracy for RESNET LISA Global pre is: 96.10%


In [118]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_global_pre.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_global.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_global_pre.pth


# Layered Pre

In [124]:
model = resnet_18(filter='None', filter_layer=0)
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [126]:
def train_layered_pruned(model, epochs):
    for iter_prune_round in range(1):
        print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
        
        # Prune layer-wise in a structured manner-
        prune.ln_structured(model.conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer1[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer1[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer1[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer1[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer1[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer1[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer1[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer1[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer2[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer2[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer2[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer2[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer2[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer2[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer2[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer2[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer3[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer3[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer3[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer3[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer3[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer3[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer3[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer3[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer4[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer4[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer4[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer4[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer4[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer4[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.layer4[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
        # prune.ln_structured(model.layer4[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)

        prune.ln_structured(model.fc, name = "weight", amount = 0.1, n = 2, dim = 0)
        
        # Print current global sparsity level-
        print(f" RESNET-18 global sparsity = {compute_sparsity_resnet(model):.2f}%")
        
        
        # Fine-training loop-
        print("\nFine-tuning pruned model to recover model's performance\n")
        model.train()
        for epoch in range(epochs):
            for x_batch, y_batch in train_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                output = model(x_batch)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()
    
            # Evaluate model!
            if epochs%10==0:
                predictions, labels = evaluate_model(model, test_dataloader, device)
                test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
                print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [128]:
model_layered_structured = train_layered_pruned(model = model.to(device), epochs = epochs)



Iterative Global pruning round = 1
 RESNET-18 global sparsity = 9.99%

Fine-tuning pruned model to recover model's performance

Epoch 1/50 - test accuracy: 62.26% and CE loss 0.84
Epoch 2/50 - test accuracy: 75.49% and CE loss 0.02
Epoch 3/50 - test accuracy: 79.28% and CE loss 0.08
Epoch 4/50 - test accuracy: 84.38% and CE loss 0.28
Epoch 5/50 - test accuracy: 91.97% and CE loss 0.00
Epoch 6/50 - test accuracy: 85.47% and CE loss 0.00
Epoch 7/50 - test accuracy: 91.32% and CE loss 0.00
Epoch 8/50 - test accuracy: 95.34% and CE loss 0.00
Epoch 9/50 - test accuracy: 94.14% and CE loss 0.00
Epoch 10/50 - test accuracy: 94.69% and CE loss 0.00
Epoch 11/50 - test accuracy: 94.90% and CE loss 0.00
Epoch 12/50 - test accuracy: 94.90% and CE loss 0.00
Epoch 13/50 - test accuracy: 94.79% and CE loss 0.00
Epoch 14/50 - test accuracy: 95.01% and CE loss 0.00
Epoch 15/50 - test accuracy: 94.90% and CE loss 0.00
Epoch 16/50 - test accuracy: 94.90% and CE loss 0.00
Epoch 17/50 - test accuracy: 94

In [129]:
# Check test set performance.
predictions, labels = evaluate_model(model_layered_structured, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for RESNET LISA Layered pre is: {(100 * test_acc):.2f}%")

Test accuracy for RESNET LISA Layered pre is: 95.01%


In [130]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_layered_pre.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_layered_structured.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_layered_pre.pth


# L1 POST

In [142]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_normal.pth"
model_save_path = model_path / model_name
model_notune = resnet_18(filter='None', filter_layer=0)
model_notune.load_state_dict(torch.load(model_save_path))
print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

RESNET-18 global sparsity = 0.00%


In [144]:
for name, module in model_notune.named_modules():
    # prune 20% of weights/connections in for all hidden layaers-
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.2)
    
    # prune 10% of weights/connections for output layer-
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.1)

print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

RESNET-18 global sparsity = 19.99%


In [146]:
model_notune.to(device)
model_notune.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_notune, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for Resnet 18 pretrained no tuning Normal is: {(100 * test_acc):.2f}%")

Test accuracy for Resnet 18 pretrained no tuning Normal is: 97.40%


In [148]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_l1_post_notune.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_notune.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_l1_post_notune.pth


In [150]:
model = model_notune.to(device)
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [152]:
def train_model(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

model_tuned = train_model(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 96.53% and CE loss 0.15
Epoch 2/50 - test accuracy: 95.44% and CE loss 0.00
Epoch 3/50 - test accuracy: 97.29% and CE loss 0.00
Epoch 4/50 - test accuracy: 97.29% and CE loss 0.00
Epoch 5/50 - test accuracy: 97.29% and CE loss 0.00
Epoch 6/50 - test accuracy: 97.18% and CE loss 0.00
Epoch 7/50 - test accuracy: 97.29% and CE loss 0.00
Epoch 8/50 - test accuracy: 97.07% and CE loss 0.00
Epoch 9/50 - test accuracy: 97.18% and CE loss 0.00
Epoch 10/50 - test accuracy: 97.07% and CE loss 0.00
Epoch 11/50 - test accuracy: 96.85% and CE loss 0.00
Epoch 12/50 - test accuracy: 97.07% and CE loss 0.00
Epoch 13/50 - test accuracy: 97.07% and CE loss 0.00
Epoch 14/50 - test accuracy: 97.07% and CE loss 0.00
Epoch 15/50 - test accuracy: 97.18% and CE loss 0.00
Epoch 16/50 - test accuracy: 96.96% and CE loss 0.00
Epoch 17/50 - test accuracy: 96.96% and CE loss 0.00
Epoch 18/50 - test accuracy: 96.96% and CE loss 0.00
Epoch 19/50 - test accuracy: 96.85% and CE loss 0.00
Ep

In [153]:
# Check test set performance.
predictions, labels = evaluate_model(model_tuned, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 96.75%


In [154]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_l1_post_tuned.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_tuned.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_l1_post_tuned.pth


# Global Post

In [156]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_normal.pth"
model_save_path = model_path / model_name
model_notune = resnet_18(filter='None', filter_layer=0)
model_notune.load_state_dict(torch.load(model_save_path))

print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

RESNET-18 global sparsity = 0.00%


In [157]:
parameters_to_prune = (
    (model_notune.conv1, 'weight'),
    (model_notune.bn1, 'weight'),
    (model_notune.layer1[0].conv1, 'weight'),
    (model_notune.layer1[0].bn1, 'weight'),
    (model_notune.layer1[0].conv2, 'weight'),
    (model_notune.layer1[0].bn2, 'weight'),
    (model_notune.layer1[1].conv1, 'weight'),
    (model_notune.layer1[1].bn1, 'weight'),
    (model_notune.layer1[1].conv2, 'weight'),
    (model_notune.layer1[1].bn2, 'weight'),
    (model_notune.layer2[0].conv1, 'weight'),
    (model_notune.layer2[0].bn1, 'weight'),
    (model_notune.layer2[0].conv2, 'weight'),
    (model_notune.layer2[0].bn2, 'weight'),
    (model_notune.layer2[1].conv1, 'weight'),
    (model_notune.layer2[1].bn1, 'weight'),
    (model_notune.layer2[1].conv2, 'weight'),
    (model_notune.layer2[1].bn2, 'weight'),
    (model_notune.layer3[0].conv1, 'weight'),
    (model_notune.layer3[0].bn1, 'weight'),
    (model_notune.layer3[0].conv2, 'weight'),
    (model_notune.layer3[0].bn2, 'weight'),
    (model_notune.layer3[1].conv1, 'weight'),
    (model_notune.layer3[1].bn1, 'weight'),
    (model_notune.layer3[1].conv2, 'weight'),
    (model_notune.layer3[1].bn2, 'weight'),
    (model_notune.layer4[0].conv1, 'weight'),
    (model_notune.layer4[0].bn1, 'weight'),
    (model_notune.layer4[0].conv2, 'weight'),
    (model_notune.layer4[0].bn2, 'weight'),
    (model_notune.layer4[1].conv1, 'weight'),
    (model_notune.layer4[1].bn1, 'weight'),
    (model_notune.layer4[1].conv2, 'weight'),
    (model_notune.layer4[1].bn2, 'weight'),
    (model_notune.fc, 'weight')
)

prune_rates_global = [0.2, 0.3, 0.4, 0.5, 0.6]

for iter_prune_round in range(1):
    print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
    
    # Prune layer-wise in a structured manner-
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method = prune.L1Unstructured,
        amount = prune_rates_global[iter_prune_round]
        
    )

    # Print current global sparsity level-
    print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")



Iterative Global pruning round = 1
RESNET-18 global sparsity = 20.00%


In [158]:
model_notune.to(device)
model_notune.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_notune, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for Resnet 18 pretrained no tuning Normal is: {(100 * test_acc):.2f}%")

Test accuracy for Resnet 18 pretrained no tuning Normal is: 97.94%


In [159]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_global_post_notune.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_notune.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_global_post_notune.pth


In [160]:
model = model_notune.to(device)
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [161]:
model_tuned = train_model(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 96.64% and CE loss 0.04
Epoch 2/50 - test accuracy: 93.71% and CE loss 0.00
Epoch 3/50 - test accuracy: 97.61% and CE loss 0.00
Epoch 4/50 - test accuracy: 93.71% and CE loss 0.00
Epoch 5/50 - test accuracy: 92.73% and CE loss 0.01
Epoch 6/50 - test accuracy: 96.53% and CE loss 0.00
Epoch 7/50 - test accuracy: 98.81% and CE loss 0.00
Epoch 8/50 - test accuracy: 98.81% and CE loss 0.00
Epoch 9/50 - test accuracy: 98.70% and CE loss 0.00
Epoch 10/50 - test accuracy: 98.81% and CE loss 0.00
Epoch 11/50 - test accuracy: 98.70% and CE loss 0.00
Epoch 12/50 - test accuracy: 98.59% and CE loss 0.00
Epoch 13/50 - test accuracy: 98.59% and CE loss 0.00
Epoch 14/50 - test accuracy: 98.59% and CE loss 0.00
Epoch 15/50 - test accuracy: 98.59% and CE loss 0.00
Epoch 16/50 - test accuracy: 98.70% and CE loss 0.00
Epoch 17/50 - test accuracy: 98.70% and CE loss 0.00
Epoch 18/50 - test accuracy: 98.70% and CE loss 0.00
Epoch 19/50 - test accuracy: 98.70% and CE loss 0.00
Ep

In [162]:
# Check test set performance.
predictions, labels = evaluate_model(model_tuned, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 98.81%


In [163]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_global_post_tuned.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_tuned.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_global_post_tuned.pth


# layered Post

In [165]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_normal.pth"
model_save_path = model_path / model_name
model_notune = resnet_18(filter='None', filter_layer=0)
model_notune.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [166]:
print(f"RESNET-18 global sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

 # Prune layer-wise in a structured manner-
prune.ln_structured(model_notune.conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer1[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer1[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer1[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer1[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer1[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer1[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer1[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer1[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer2[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer2[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer2[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer2[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer2[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer2[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer2[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer2[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer3[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer3[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer3[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer3[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer3[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer3[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer3[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer3[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer4[0].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer4[0].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer4[0].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer4[0].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer4[1].conv1, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer4[1].bn1, name = "weight", amount = 0.1, n = 2, dim = 0)
prune.ln_structured(model_notune.layer4[1].conv2, name = "weight", amount = 0.1, n = 2, dim = 0)
# prune.ln_structured(model.layer4[1].bn2, name = "weight", amount = 0.1, n = 2, dim = 0)

prune.ln_structured(model_notune.fc, name = "weight", amount = 0.1, n = 2, dim = 0)

# Print current global sparsity level-
print(f"Resnet structrured sparsity = {compute_sparsity_resnet(model_notune):.2f}%")

RESNET-18 global sparsity = 0.00%
Resnet structrured sparsity = 9.99%


In [167]:
model_notune.to(device)
model_notune.eval()


# Check test set performance.
predictions, labels = evaluate_model(model_notune, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Test accuracy for Resnet 18 pretrained no tuning Normal is: {(100 * test_acc):.2f}%")

Test accuracy for Resnet 18 pretrained no tuning Normal is: 89.05%


In [168]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_layered_post_notune.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_notune.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_layered_post_notune.pth


In [169]:
model = model_notune.to(device)
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model.parameters(), lr = learning_rate, eps = 1e-06)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [170]:
model_tuned = train_model(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 92.95% and CE loss 0.40
Epoch 2/50 - test accuracy: 94.79% and CE loss 0.00
Epoch 3/50 - test accuracy: 91.21% and CE loss 0.01
Epoch 4/50 - test accuracy: 94.03% and CE loss 0.00
Epoch 5/50 - test accuracy: 96.96% and CE loss 0.00
Epoch 6/50 - test accuracy: 87.74% and CE loss 0.00
Epoch 7/50 - test accuracy: 96.53% and CE loss 0.00
Epoch 8/50 - test accuracy: 99.02% and CE loss 0.00
Epoch 9/50 - test accuracy: 98.92% and CE loss 0.00
Epoch 10/50 - test accuracy: 98.81% and CE loss 0.00
Epoch 11/50 - test accuracy: 99.02% and CE loss 0.00
Epoch 12/50 - test accuracy: 99.02% and CE loss 0.00
Epoch 13/50 - test accuracy: 99.13% and CE loss 0.00
Epoch 14/50 - test accuracy: 99.13% and CE loss 0.00
Epoch 15/50 - test accuracy: 99.13% and CE loss 0.00
Epoch 16/50 - test accuracy: 99.13% and CE loss 0.00
Epoch 17/50 - test accuracy: 99.13% and CE loss 0.00
Epoch 18/50 - test accuracy: 99.24% and CE loss 0.00
Epoch 19/50 - test accuracy: 99.24% and CE loss 0.00
Ep

In [171]:
# Check test set performance.
predictions, labels = evaluate_model(model_tuned, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 99.35%


In [172]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "resnet_lisa_layered_post_tuned.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_tuned.state_dict(), f=model_save_path)

Saving the model: models\resnet_lisa_layered_post_tuned.pth
